In [2]:
import numpy as np

In [49]:
a = 0.125
h = 1
V = 0.5
item = np.array([], dtype=np.int32).reshape(0,4)
for i in range(4):
    for j in range(4):
        xi = -0.5 + a*(1+2*i)
        yj = -0.5 + a*(1+2*j)
        temp = np.array([xi,yj, h, V]).reshape(1,4)
        item = np.concatenate((item,temp))

h = 2
V = -0.5
for i in range(4):
    for j in range(4):
        xi = -0.5 + a*(1+2*i)
        yj = -0.5 + a*(1+2*j)
        temp = np.array([xi,yj, h, V]).reshape(1,4)
        item = np.concatenate((item,temp))
item

array([[-0.375, -0.375,  1.   ,  0.5  ],
       [-0.375, -0.125,  1.   ,  0.5  ],
       [-0.375,  0.125,  1.   ,  0.5  ],
       [-0.375,  0.375,  1.   ,  0.5  ],
       [-0.125, -0.375,  1.   ,  0.5  ],
       [-0.125, -0.125,  1.   ,  0.5  ],
       [-0.125,  0.125,  1.   ,  0.5  ],
       [-0.125,  0.375,  1.   ,  0.5  ],
       [ 0.125, -0.375,  1.   ,  0.5  ],
       [ 0.125, -0.125,  1.   ,  0.5  ],
       [ 0.125,  0.125,  1.   ,  0.5  ],
       [ 0.125,  0.375,  1.   ,  0.5  ],
       [ 0.375, -0.375,  1.   ,  0.5  ],
       [ 0.375, -0.125,  1.   ,  0.5  ],
       [ 0.375,  0.125,  1.   ,  0.5  ],
       [ 0.375,  0.375,  1.   ,  0.5  ],
       [-0.375, -0.375,  2.   , -0.5  ],
       [-0.375, -0.125,  2.   , -0.5  ],
       [-0.375,  0.125,  2.   , -0.5  ],
       [-0.375,  0.375,  2.   , -0.5  ],
       [-0.125, -0.375,  2.   , -0.5  ],
       [-0.125, -0.125,  2.   , -0.5  ],
       [-0.125,  0.125,  2.   , -0.5  ],
       [-0.125,  0.375,  2.   , -0.5  ],
       [ 0.125, 

$$
L_{i,j}\approx \frac{k}{r + [L_a \cdot (1 + r/a)]^{-1}}
$$
where $a=b$ (squares), and
$$
r = \sqrt{(x_i-x_j)^2 + (y_i-y_j)^2 + (z_i-z_j)^2}
$$